In [419]:
import os
from collections import namedtuple
from astropy.io import fits
from astropy.io import ascii as asc
from astropy.table import Table
from astropy.modeling import models,fitting
from astropy.convolution import convolve, Box1DKernel
import numpy as np
from scipy import signal, interpolate

from matplotlib import pyplot as plt
import matplotlib.collections as collections
%matplotlib
import spectroscopy as spec

Using matplotlib backend: Qt5Agg


In [425]:
def read_iraf_spectrum(filename, flux_loc=0):
    ofile = fits.open(filename)
    flux = ofile[0].data[flux_loc,0,:]
    err = ofile[0].data[3,0,:]
    wave = spec.calc_wavelength(ofile[0].header, np.arange(len(flux))+1)
    rest_wave = spec.apply_redshift(wave, 0.0069)
    return(spec.spectrum1d(rest_wave, flux, error=err))

In [383]:
DATA_DIR_LCO = '../data/spectra/lco'
DATA_DIR_EFOSC = '../data/spectra/EFOSC'

In [384]:
spectra_files = [
         ('asassn15oz_20150904_redblu_122216.314.fits', DATA_DIR_LCO),
         ('asassn-15oz_20150906_redblu_105042.698a.fits', DATA_DIR_LCO),
         ('asassn15oz_20150907_redblu_123835.277.fits', DATA_DIR_LCO),
         ('asassn15oz_20150911_redblu_105336.349.fits', DATA_DIR_LCO),
         ('asassn15oz_20150916_redblu_120911.274.fits', DATA_DIR_LCO),
         ('asassn-15oz_20150920_redblu_135034.512.fits',DATA_DIR_LCO),
         ('asassn-15oz_20150924_redblu_123847.580.fits',DATA_DIR_LCO),
         ('asassn-15oz_20150930_redblu_122858.217.fits',DATA_DIR_LCO),
         ('tASASSN-15oz_20151003_Gr13_Free_slit1.0_57720_1_e.fits',DATA_DIR_EFOSC),
         ('asassn15oz_20151006_redblu_101906.800.fits', DATA_DIR_LCO),
         ('asassn15oz_20151014_redblu_112918.305.fits', DATA_DIR_LCO),
         ('asassn-15oz_20151025_redblu_102221.833.fits', DATA_DIR_LCO),
         ('asassn-15oz_20151107_redblu_101210.833.fits', DATA_DIR_LCO),
         ('tASAS-SN_15oz_20151107_Gr13_Free_slit1.5_57723_1_e.fits', DATA_DIR_EFOSC),
         ('tASAS-SN_15oz_20151118_Gr13_Free_slit1.0_57723_1_e.fits', DATA_DIR_EFOSC)
                ]

In [385]:
ifile, idir = spectra_files[10]
spectrum = read_iraf_spectrum(os.path.join(idir, ifile))

# Get a sense of the smoothing function

In [386]:
smooth_flux1 = signal.savgol_filter(spectrum.flux, 21, 4)
smooth_flux2 = signal.savgol_filter(spectrum.flux, 5, 2)

In [387]:
#I think that a larger width will introduce more smoothing and that a lower order polynomial will introduce more smoothing
fig = plt.figure()
wave_range = [(3900, 4100), (7000, 7200), (9500, 9700)]
flux = [spectrum.flux, smooth_flux1, smooth_flux2]
plot_num = 1
ax_list = []
for iflux in flux:
    for wmin, wmax in wave_range:
        windx = (spectrum.wave<wmax) & (spectrum.wave>wmin)
        ax = fig.add_subplot(3,3,plot_num)
        ax_list.append(ax)
        ax.plot(spectrum.wave[windx], iflux[windx])
        plot_num+=1
fig.tight_layout()

# Find the Edges

In [388]:
def smooth_signal(flux, width, poly_deg):
    smoothed_flux = signal.savgol_filter(flux, width, poly_deg)
    return smoothed_flux
    
def find_blue_edge(wave, flux, wcenter, binsize, wmin=None):
    '''
    Calculate the slope in each bin starting at wmin, until the bin changes sign, use center for blue_edge
    '''
    wcenter_indx = np.argmin(np.abs(wave-wcenter))
    ifit = np.polyfit(wave[wcenter_indx-binsize: wcenter_indx+1],flux[wcenter_indx-binsize: wcenter_indx+1], 1)
    slope_product = 1
    #plt.plot(wave, flux)
    #plt.xlim(wmin, wcenter)
    if wmin is None:
        search_indx = np.arange(binsize,wcenter_indx+1)
    else:
        min_indx = np.argmin(np.abs(wave - wmin))
        search_indx = np.arange(min_indx, wcenter_indx)
    for indx in search_indx[::-1]:
        last_slope = ifit[0]
        if indx-binsize < 0:
            break
        ifit = np.polyfit(wave[indx-binsize:indx+1], flux[indx-binsize:indx+1], 1)
        #plt.plot(wave[indx-binsize:indx+1], flux[indx-binsize:indx+1])
        #plt.plot(wave[indx-binsize:indx+1], np.polyval(ifit, wave[indx-binsize:indx+1]))
        slope_product = last_slope*ifit[0] #if this is negative then the slope has changed sign
        if slope_product < 0:
            break  
    edge_indx = indx - binsize//2
    return edge_indx, wave[edge_indx] 

    
def find_red_edge(wave, flux, wcenter, binsize, wmax = None):
    '''
    Calculate the slope in each bin starting at wmin, until the bin changes sign, use center for red_edge
    binsize is in pixels
    '''
    wcenter_indx = np.argmin(np.abs(wave-wcenter))
    #fig = plt.figure()
    #ax1 = fig.add_subplot(2,1,1)
    #ax2 = fig.add_subplot(2,1,2, sharex=ax1)
    #ax1.plot(wave, flux)
    #ax2.plot(wave, wave-wcenter)
    ifit = np.polyfit(wave[wcenter_indx:wcenter_indx+binsize+1],flux[wcenter_indx:wcenter_indx+binsize+1], 1)
    slope_product = 1
    #plt.plot(wave, flux)
    #plt.xlim(wmin, wcenter)
    #plt.axvline(wave[wcenter_indx], color='y')
    if wmax is None:
        search_indx = np.arange(wcenter_indx, len(flux))
    else:
        max_indx = np.argmin(np.abs(wave - wmax))
        search_indx = np.arange(wcenter_indx, max_indx+1)
    #plt.plot(wave[search_indx], flux[search_indx])
    for indx in search_indx:
        last_slope = ifit[0]
        ifit = np.polyfit(wave[indx:indx+binsize+1], flux[indx:indx+binsize+1], 1)
        slope_product = last_slope*ifit[0] #if this is negative then the slope has changed sign
        #plt.plot(wave[indx:indx+binsize+1], flux[indx:indx+binsize+1])
        #plt.plot(wave[indx:indx+binsize+1], np.polyval(ifit, wave[indx:indx+binsize+1]))
        slope_product = last_slope*ifit[0] #if this is negative then the slope has changed sign
        if slope_product < 0:
            break  
    edge_indx = indx + binsize//2
    return edge_indx, wave[edge_indx] 
    
def check_max(wave, flux, edge_indx, binsize, absorption=True):
    '''
    Fit a quadratic and verify that it is the correct direction
    '''
    wmin_indx = edge_indx - binsize//2
    wmax_indx = edge_indx + binsize//2
    fit = np.polyfit(wave[wmin_indx:wmax_indx+1], flux[wmin_indx:wmax_indx+1], 2)
    if fit[0]>0:
        concavity = 'up'
    if fit[0]<0:
        concavity = 'down'
    if ((absorption is True) and (concavity is 'down')) or ((absorption is False) and (concavity is 'up')):
        good_fit = True
    else:
        good_fit = False
    return good_fit



In [389]:
def calc_rmse(data, model):
    rmse = np.sqrt(np.sum((model-data)**2)/len(data))
    print('rmse calculated over {} points'.format(len(data)))
    return rmse
def test_calc_rmse1():
    data = np.ones(10)
    model = np.ones(10)
    rmse = calc_rmse(data, model)
    assert np.isclose(rmse,0)
def test_calc_rmse2():
    data = np.ones(10)
    model = np.zeros(10)
    rmse = calc_rmse(data, model)
    assert np.isclose(rmse,1)
test_calc_rmse1()
test_calc_rmse2()

rmse calculated over 10 points
rmse calculated over 10 points


In [390]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 21
binsize=21
#wmin = 5400
#wmax = 5700
wmin = None
wmax = None
wcenter=5550

smooth_flux= smooth_signal(flux, width, poly_deg)
blue_edge_indx, blue_edge_wl = find_blue_edge(wave, smooth_flux, wcenter, binsize, wmin=wmin)
red_edge_indx, red_edge_wl = find_red_edge(wave, smooth_flux, wcenter, binsize, wmax = wmax)
if blue_edge_indx is not None:
    good_fit_blue = check_max(wave, smooth_flux, blue_edge_indx, binsize, absorption=True)
if red_edge_indx is not None:
    good_fit_red = check_max(wave, smooth_flux, red_edge_indx, binsize, absorption=True)

In [391]:
print(blue_edge_indx, blue_edge_wl)
print(red_edge_indx, red_edge_wl)
print(good_fit_blue, good_fit_red)
print( flux.shape)

1211 5465.03886138
1292 5604.74536071
True True
(3802,)


In [393]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
l2 = plt.plot(wave, smooth_flux, color='r')
plt.axvline(wcenter, linestyle = ':', color = 'y', label='line_center')
plt.xlim(wmin, wmax)
plt.axvline(blue_edge_wl, color='b', ls='--')
plt.axvline(red_edge_wl, color='r', ls='--')

# Refactor finding the edges

The previous functions are very subjective to where you put the line center. It should be that if you give the function a wavelength range that includes the center and the edges and your binsize is sufficiently large then there should be 3 places where the slope changes sign that correspond to the blue edge, the center, and the red edge. This method is much more robust and does not require an accurate guess of the location of the minimum. Perhaps keep this one around for finding the edge of a blended line?

In [394]:
def find_boundary(wave, flux, wmin, wmax, binsize, visualize=False):
    wmin_indx = np.argmin(np.abs(wave-wmin))
    wmax_indx = np.argmin(np.abs(wave-wmax))
    slope_product = []
    ifit = np.polyfit(wave[wmin_indx-binsize//2:wmin_indx+binsize//2+1], flux[wmin_indx-binsize//2:wmin_indx+binsize//2+1], 1)
    search_indx = np.arange(wmin_indx, wmax_indx+1)
    if visualize:
        fig = plt.figure()
        ax1 = fig.add_subplot(2,1,1)
        ax2 = fig.add_subplot(2,1,2, sharex=ax1)
        ax1.set_xlim(wmin, wmax)
    for indx in search_indx:
        last_slope = ifit[0]
        ifit = np.polyfit(wave[indx-binsize//2:indx+binsize//2+1], flux[indx-binsize//2:indx+binsize//2+1], 1)
        slope_product.append(last_slope*ifit[0]) #if this is negative then the slope has changed sign
        if visualize:
            ax1.plot(wave[indx-binsize//2:indx+binsize//2+1], np.polyval(ifit, wave[indx-binsize//2:indx+binsize//2+1]))
    slope_product = np.array(slope_product)
    slope_change_indx = search_indx[slope_product<0]
    if visualize:
        ax1.plot(wave, flux)
        ax2.plot(wave[search_indx], slope_product)
        ax2.axhline(0, color='k', linestyle=':')
    if len(slope_change_indx) == 3:
        blue_edge_indx, wcenter_indx, red_edge_indx = slope_change_indx
        return blue_edge_indx, wcenter_indx, red_edge_indx
    else:
        return None, None, None

In [395]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=15
wmin = 5400
wmax = 5700

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize)
if blue_edge_indx is not None:
    good_fit_blue = check_max(wave, smooth_flux, blue_edge_indx, binsize, absorption=True)
    print(good_fit_blue)
else:
    print(blue_edge_not_found)
if red_edge_indx is not None:
    good_fit_red = check_max(wave, smooth_flux, red_edge_indx, binsize, absorption=True)
    print(good_fit_red)
else:
    print(red_edge_not_found)

True
True


In [396]:
wcenter = wave[wcenter_indx]
blue_edge_wl = wave[blue_edge_indx]
red_edge_wl = wave[red_edge_indx]
plt.axvspan(wmin, wmax, color='y', alpha = 0.25, label='fit_range')
plt.plot(wave, flux)
plt.plot(wave, smooth_flux, '.')
plt.axvline(wcenter, linestyle = ':', color = 'y', label='line_center')
plt.xlim(wmin, wmax)
plt.axvline(blue_edge_wl, color='b', ls='--')
plt.axvline(red_edge_wl, color='r', ls='--')

# Define Continuum

In [399]:
endpoint = namedtuple('endpoint', ['wave', 'flux', 'error'])
def determine_error_binsize(wave, wave_bin=100):
    '''
    We should be calculating noise over the same wavelength range
    rather than the same number of pixels as long as one wavelength
    bin includes enough pixels. Set binsize to be 100A. If there are
    fewer than 10 pixels in 100A (dispersion is greater than 10A/pix)
    then issue a warning and make binsize 10 pixels regardless of how
    many angstroms this represents
    
    wave: array of wavelengths
    wave_bin: size of bin in angstroms
    
    outputs: binsize in pixels
    
    Note: right now this calculates the dispersion for the full wavelength range.
    For a grating/grism with a large variation in dispersion, it might make sense to
    just calculate this over the feature wavelength range.
    '''
    dispersion = np.median(wave[1:]-wave[:-1])
    binsize = np.ceil(wave_bin/dispersion)
    if binsize < 10:
        print('WARNING: dispersion = {}, \
        leading to binsize < 10 for {}$\AA$ bins, \
        setting binsize=10, making wave_bin={}'.format(dispersion, wave_bin, 10*dispersion))
        binsize=10
    return binsize
def define_continuum(wave, flux, edge_indx, binsize, err_binsize, absorption=True, visualize=False):
    '''
    Fit a quadratic and verify that it is the correct direction
    '''
    #Silverman says: "Once these two endpoints are determined, a quadratic function is 
    # fit to the data in wavelength bins centred on each endpoint." 
    #Let's start with fitting over 2 wavelength bins?
    wmin_indx = edge_indx - int(np.floor(1*binsize))
    wmax_indx = edge_indx + int(np.ceil(1*binsize))
    quad_model = models.Polynomial1D(degree=2)
    fitter = fitting.LinearLSQFitter()
    fit = fitter(quad_model, wave[wmin_indx:wmax_indx+1], flux[wmin_indx:wmax_indx+1])
    fit_extreme_wl = -fit.c1.value/(2*fit.c2.value)

    #calc rmse over edge_indx +/- 20 pixels
    wmin_rmse = edge_indx - int(err_binsize//2)
    wmax_rmse = edge_indx + int(err_binsize//2)
    rmse = calc_rmse(flux[wmin_rmse:wmax_rmse], fit(wave[wmin_rmse:wmax_rmse]))
    
    if fit.c2.value>0:
        concavity = 'up'
    if fit.c2.value<0:
        concavity = 'down'
    if ((absorption is True) and (concavity is 'down')) or ((absorption is False) and (concavity is 'up')):
        good_fit = True
    else:
        good_fit = False
    if visualize:
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.plot(wave, flux)
        ax.set_xlim(wave[edge_indx-4*binsize], wave[edge_indx+4*binsize])
        ax.plot(wave[wmin_indx-10:wmax_indx+11], fit(wave[wmin_indx-10:wmax_indx+11]))
        ax.plot(wave[wmin_indx:wmax_indx+1], fit(wave[wmin_indx:wmax_indx+1]), label='fit range')
        ax.axvline(wave[edge_indx], label='input continuum', color='k', ls=':')
        ax.errorbar(fit_extreme_wl, fit(fit_extreme_wl), yerr=rmse, fmt='.', label='Edge w/error', zorder=10 )
        ax.set_ylim(0.95*np.min(flux[edge_indx-4*binsize:edge_indx+4*binsize]), 1.05*np.max(flux[edge_indx-4*binsize:edge_indx+4*binsize]))
        ax.legend(loc='best')
    endpt = endpoint(fit_extreme_wl, fit(fit_extreme_wl), rmse)
    return good_fit, endpt

In [400]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=15
wmin = 5400
wmax = 5700

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize)
err_binsize = determine_error_binsize(wave)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize, absorption=True, visualize=True)
    print(good_fit_blue)
else:
    print(blue_edge_not_found)
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize, absorption=True, visualize=True)
    print(good_fit_red)
else:
    print(red_edge_not_found)

rmse calculated over 58 points
True
rmse calculated over 58 points
True


In [401]:
wcenter = wave[wcenter_indx]
blue_edge_wl = wave[blue_edge_indx]
red_edge_wl = wave[red_edge_indx]
plt.figure()
plt.axvspan(wmin, wmax, color='y', alpha = 0.1, label='fit_range')
plt.plot(wave, flux)
plt.plot(wave, smooth_flux, '.')
plt.axvline(wcenter, linestyle = ':', color = 'y', label='line_center')
plt.xlim(wmin, wmax)
plt.axvline(blue_edge_wl, color='b', ls='--', alpha=0.5)
plt.axvline(red_edge_wl, color='r', ls='--', alpha=0.5)
plt.plot(continuum_l.wave, continuum_l.flux, color='b', marker='*')
plt.plot(continuum_r.wave, continuum_r.flux, color='r', marker='*')

# Calculate EW

Silverman define pseudo equivalent widths as:

$pEW = \sum_{i=0}^{i=N-1}\Delta \lambda_{i} \frac{f_c(\lambda_{i}) - f(\lambda_{i})}{f_c(\lambda_{i})}$

where $f_c$ is the continuum, f is the flux, i is the pixel number and N is the number of pixels between the blue and red edge

In [402]:
def calc_pseudo_ew(wave, flux, continuum_l, continuum_r, absorption=True, visualize=False):
    fitter = fitting.LinearLSQFitter()
    lin_mod = models.Linear1D()
    continuum_fit = fitter(lin_mod,[continuum_l.wave, continuum_r.wave], [continuum_l.flux, continuum_r.flux])
    line_indx = np.int_(np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)])
    continuum = continuum_fit(wave[line_indx])
    delta_lambda = wave[line_indx]-wave[line_indx-1]
    if absorption is True:
        pew = np.sum(delta_lambda*(continuum - flux[line_indx])/continuum)
    else:
        pew = np.sum(delta_lambda*(flux[line_indx]-continuum)/continuum) #Check that this is true
    if visualize is True:
        fig = plt.figure()
        ax1 = fig.add_subplot(1,2,1)
        ax2 = fig.add_subplot(1,2,2)
        ax2.axhline(1, color='k')
        ax1.plot(wave, flux)
        ax1.plot(wave[line_indx], flux[line_indx], label='data')
        ax1.plot(wave[line_indx], continuum, label='continuum')
        ax1.set_xlim(continuum_l.wave-10, continuum_r.wave+10)
        if absorption is True:
            ax2.plot(wave[line_indx], (continuum - flux[line_indx])/continuum, label='sum for pEW')
        else:
            ax2.plot(wave[line_indx], (flux[line_indx]-continuum)/continuum, label='sum for pEW')
    return pew

In [403]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=15
wmin = 5400
wmax = 5700

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize)
err_binsize = determine_error_binsize(wave)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize, absorption=True, visualize=True)
    print(good_fit_blue)
else:
    print(blue_edge_not_found)
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize, absorption=True, visualize=True)
    print(good_fit_red)
else:
    print(red_edge_not_found)
pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=True)

rmse calculated over 58 points
True
rmse calculated over 58 points
True


In [404]:
def calc_continuum(wave, continuum_l, continuum_r):
    fitter = fitting.LinearLSQFitter()
    lin_mod = models.Linear1D()
    continuum_fit = fitter(lin_mod,[continuum_l.wave, continuum_r.wave], [continuum_l.flux, continuum_r.flux])
    continuum = continuum_fit(wave)
    return continuum

In [405]:
wcenter = wave[wcenter_indx]
blue_edge_wl = wave[blue_edge_indx]
red_edge_wl = wave[red_edge_indx]
fig = plt.figure(figsize=[12, 5])
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax1.axvspan(wmin, wmax, color='y', alpha = 0.1, label='fit_range')
ax1.plot(wave, flux, label='data')
ax1.plot(wave, smooth_flux, '.', label='smoothed flux')
ax1.set_xlim(wmin-15, wmax+15)
ax1.set_ylim(flux[blue_edge_indx+red_edge_indx+1].min(), flux[blue_edge_indx:red_edge_indx+1].max())
ax1.plot(continuum_l.wave, continuum_l.flux, color='k', marker='+', markersize=8, mew=2, label='Continuum', ls='none')
ax1.plot(continuum_r.wave, continuum_r.flux, color='k', marker='+', markersize=8, mew=2, ls='none')
ax1.plot(wcenter)
ax1.legend(loc='lower left')
ax1.set_xlabel('Wavelength')
ax1.set_ylabel('Flux')
line_indx = np.int_(np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)])
continuum = calc_continuum(wave[line_indx], continuum_l, continuum_r)

ax2.plot(wave[line_indx], flux[line_indx]/continuum, label='normalized flux')
ax2.axvspan(wcenter-pew/2, wcenter+pew/2, label='pEW', color='k', alpha=0.1)
ax2.set_ylim(0,1.1)
ax2.axhline(1, color='k', alpha=0.5, linestyle=':')
ax2.legend(loc='lower left')
ax2.set_xlabel('Wavelength')
ax2.set_ylabel('Normalized Flux')

Text(0,0.5,'Normalized Flux')

## Try a different line:

In [406]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=21
wmin = 4600
wmax = 4875

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize, visualize=False)
err_binsize = determine_error_binsize(wave)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize, absorption=True, visualize=True)
    print(good_fit_blue)
else:
    print('blue_edge_not_found')
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize, absorption=True, visualize=True)
    print(good_fit_red)
else:
    print('red_edge_not_found')
if (blue_edge_indx is not None) and (red_edge_indx is not None):
    pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=True)

rmse calculated over 58 points
True
rmse calculated over 58 points
True


In [407]:
wcenter = wave[wcenter_indx]
blue_edge_wl = wave[blue_edge_indx]
red_edge_wl = wave[red_edge_indx]
fig = plt.figure(figsize=[12, 5])
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax1.axvspan(wmin, wmax, color='y', alpha = 0.1, label='fit_range')
ax1.plot(wave, flux, label='data')
ax1.plot(wave, smooth_flux, '.', label='smoothed flux')
ax1.set_xlim(wmin-15, wmax+15)
ax1.set_ylim(flux[blue_edge_indx:red_edge_indx+1].min(), flux[blue_edge_indx:red_edge_indx+1].max())
ax1.plot(continuum_l.wave, continuum_l.flux, color='k', marker='+', markersize=8, mew=2, label='Continuum', ls='none')
ax1.plot(continuum_r.wave, continuum_r.flux, color='k', marker='+', markersize=8, mew=2, ls='none')
ax1.plot(wcenter)
ax1.legend(loc='lower left')
ax1.set_xlabel('Wavelength')
ax1.set_ylabel('Flux')

line_indx = np.int_(np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)])
continuum = calc_continuum(wave[line_indx], continuum_l, continuum_r)

ax2.plot(wave[line_indx], flux[line_indx]/continuum, label='normalized flux')
ax2.axvspan(wcenter-pew/2, wcenter+pew/2, label='pEW', color='k', alpha=0.1)
ax2.set_ylim(0,1.1)
ax2.axhline(1, color='k', alpha=0.5, linestyle=':')
ax2.legend(loc='lower left')
ax2.set_xlabel('Wavelength')
ax2.set_ylabel('Normalized Flux')

Text(0,0.5,'Normalized Flux')

# Testing different types of fits for the velocity

In [408]:
def test_find_velocity(wave, flux, wcenter, continuum_l, continuum_r, fit_func=interpolate.UnivariateSpline, visualization=False):
    '''
    We explored many different ways of fitting (Polynomial up to degree 6, UnivariateSpline (setting s, deg 3 and 4), 
    and LSQUnivariateSpline (setting the knots to be the edges of the feature + the minimum or the halfway point).
    Without using the weight by the error, I couldn't get a good fit out of UnivariateSpline (it always fit with 2 knots
    looking like a polynomial).
    '''
    line_indx = np.int_(np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)])
    fitter = fitting.LinearLSQFitter()
    lin_mod = models.Linear1D()
    flux = flux
    continuum_fit = fitter(lin_mod,[continuum_l.wave, continuum_r.wave], [continuum_l.flux, continuum_r.flux])
    continuum = continuum_fit(wave[line_indx])
    weight= 1./spectrum.error
    x = wave[line_indx[0]-20:line_indx[-1]+20]
    w = weight[line_indx[0]-20:line_indx[-1]+20] 
    y = (flux[line_indx[0]-20:line_indx[-1]+20]-continuum_fit(wave[line_indx[0]-20:line_indx[-1]+20]))
    fit = interpolate.UnivariateSpline(x, 
                   y, 
                   w=w,
                   k=3)
    
    spl = fit
    print('s', np.sum((w * (y-spl(x)))**2, axis=0))
    #fit2 = fitter(models.Polynomial1D(degree=6), wave[line_indx[0]-20:line_indx[-1]+20], flux[line_indx[0]-20:line_indx[-1]+20]-continuum_fit(wave[line_indx[0]-20:line_indx[-1]+20]))
    #knots = [continuum_l.wave, wave[line_indx][flux[line_indx].argmin()], continuum_r.wave]
    #knots = [continuum_l.wave, np.mean([continuum_l.wave, continuum_r.wave]), continuum_r.wave]
    
    #fit3 = interpolate.LSQUnivariateSpline(wave[line_indx[0]-20:line_indx[-1]+20], flux[line_indx[0]-20:line_indx[-1]+20]-continuum_fit(wave[line_indx[0]-20:line_indx[-1]+20]), t=knots)
    smooth_flux = smooth_signal(flux[line_indx[0]-20:line_indx[-1]+20]-continuum_fit(wave[line_indx[0]-20:line_indx[-1]+20]),51, 2)
    if visualization is True:
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.plot(wave[line_indx[0]-20:line_indx[-1]+20], flux[line_indx[0]-20:line_indx[-1]+20]-continuum_fit(wave[line_indx[0]-20:line_indx[-1]+20]))
        ax.plot(wave[line_indx[0]-20:line_indx[-1]+20], fit(wave[line_indx[0]-20:line_indx[-1]+20]), label='spline,default knots, 3deg')
        #ax.plot(wave[line_indx[0]-20:line_indx[-1]+20], fit2(wave[line_indx[0]-20:line_indx[-1]+20]), label='6deg Poly')
        #ax.plot(wave[line_indx[0]-20:line_indx[-1]+20], fit3(wave[line_indx[0]-20:line_indx[-1]+20]), label='spline, 3 knots, 3deg')
        ax.plot(wave[line_indx[0]-20:line_indx[-1]+20], smooth_flux, label='Smoothed, binsize=51', color='m')
        min_wave = wave[line_indx][np.argmin(fit(wave[line_indx]))]
        ax.axvline(min_wave, color='k', linestyle='--', label='min')
        #ax.axvline(wave[line_indx][np.argmin(fit3(wave[line_indx]))], color='r', linestyle='--', label='min')
        knots1 = fit.get_knots()
        ax.vlines(knots1, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], linestyle=':', label='spline, default knots location')
        #ax.vlines(knots, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], linestyle=':', color='r', label='spline, 3knots location')
        ax.legend(loc='best', fontsize='x-small')
    return fit

In [410]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=21
wmin = 4600
wmax = 4875
fitting_function = interpolate.UnivariateSpline
#fitting_function = interpolate.LSQUnivariateSpline

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize, visualize=False)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize,absorption=True, visualize=False)
    print(good_fit_blue)
else:
    print('blue_edge_not_found')
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize,absorption=True, visualize=False)
    print(good_fit_red)
else:
    print('red_edge_not_found')
if (blue_edge_indx is not None) and (red_edge_indx is not None):
    pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=False)
    wcenter = wave[wcenter_indx]
    vel_fit = test_find_velocity(wave, smooth_flux, wcenter, continuum_l, continuum_r, visualization=True)


rmse calculated over 58 points
True
rmse calculated over 58 points
True
s 184.959995059


In [411]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=15
wmin = 5400
wmax = 5700
fitting_function = interpolate.UnivariateSpline

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize, visualize=False)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize,err_binsize, absorption=True, visualize=False)
    print(good_fit_blue)
else:
    print('blue_edge_not_found')
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize,err_binsize, absorption=True, visualize=False)
    print(good_fit_red)
else:
    print('red_edge_not_found')
if (blue_edge_indx is not None) and (red_edge_indx is not None):
    pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=False)
    wcenter = wave[wcenter_indx]
    vel_fit = test_find_velocity(wave, smooth_flux, wcenter, continuum_l, continuum_r, visualization=True)

rmse calculated over 58 points
True
rmse calculated over 58 points
True
s 120.948005382


In [413]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=37
wmin = 4925
wmax = 5375
fitting_function = interpolate.UnivariateSpline

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize, visualize=False)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize,absorption=True, visualize=True)
    print(good_fit_blue)
else:
    print('blue_edge_not_found')
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize,absorption=True, visualize=True)
    print(good_fit_red)
else:
    print('red_edge_not_found')
if (blue_edge_indx is not None) and (red_edge_indx is not None):
    pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=False)
    wcenter = wave[wcenter_indx]
    vel_fit = test_find_velocity(wave, smooth_flux, wcenter, continuum_l, continuum_r, visualization=True)

rmse calculated over 58 points
True
rmse calculated over 58 points
True
s 268.840336707


Conclusion: Use UnivariateSpline with weights of 1/stddev. If no error column is present, use a heavily smoothed function to approximate the continuum and calculate the standard deviation from that as the error

# Fit the Velocity

In [414]:
def find_velocity(wave, flux, wcenter, continuum_l, continuum_r, binsize, visualization=False):
    line_indx = np.int_(np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)])
    windx_min = line_indx[0]-binsize//2
    windx_max = line_indx[-1]+binsize//2
    fitter = fitting.LinearLSQFitter()
    lin_mod = models.Linear1D()
    continuum_fit = fitter(lin_mod,[continuum_l.wave, continuum_r.wave], [continuum_l.flux, continuum_r.flux])
    continuum = continuum_fit(wave[windx_min:windx_max])
    weight = 1./spectrum.error[windx_min:windx_max]
    fit = interpolate.UnivariateSpline(wave[windx_min:windx_max], flux[windx_min:windx_max]-continuum, w=weight)
    if visualization is True:
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.plot(wave[windx_min:windx_max], flux[windx_min:windx_max]-continuum)
        ax.plot(wave[windx_min:windx_max], fit(wave[windx_min:windx_max]))
        min_wave = wave[line_indx][np.argmin(fit(wave[line_indx]))]
        ax.axvline(min_wave)
        knots = fit.get_knots()
        ax.vlines(knots, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], linestyle=':')
    return fit

In [415]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=21
wmin = 4600
wmax = 4875

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize, visualize=False)
err_binsize = determine_error_binsize(wave, wave_bin=100)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize, absorption=True, visualize=False)
    print(good_fit_blue)
else:
    print('blue_edge_not_found')
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize, absorption=True, visualize=False)
    print(good_fit_red)
else:
    print('red_edge_not_found')
if (blue_edge_indx is not None) and (red_edge_indx is not None):
    pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=False)
    wcenter = wave[wcenter_indx]
    vel_fit = find_velocity(wave, smooth_flux, wcenter, continuum_l, continuum_r, err_binsize, visualization=True)

rmse calculated over 58 points
True
rmse calculated over 58 points
True


/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [416]:
wcenter = wave[wcenter_indx]
blue_edge_wl = wave[blue_edge_indx]
red_edge_wl = wave[red_edge_indx]
fig = plt.figure(figsize=[12, 5])
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax1.axvspan(wmin, wmax, color='y', alpha = 0.1, label='fit_range')
ax1.plot(wave, flux, label='data')
ax1.plot(wave, smooth_flux, '.', label='smoothed flux')
ax1.set_xlim(wmin-15, wmax+15)
ax1.set_ylim(flux[blue_edge_indx:red_edge_indx+1].min(), flux[blue_edge_indx:red_edge_indx+1].max())
ax1.plot(continuum_l.wave, continuum_l.flux, color='k', marker='+', markersize=8, mew=2, label='Continuum', ls='none')
ax1.plot(continuum_r.wave, continuum_r.flux, color='k', marker='+', markersize=8, mew=2, ls='none')


line_indx = np.int_(np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)])
fitter = fitting.LinearLSQFitter()
lin_mod = models.Linear1D()
continuum_fit = fitter(lin_mod,[continuum_l.wave, continuum_r.wave], [continuum_l.flux, continuum_r.flux])
continuum = continuum_fit(wave[line_indx])

min_vel_indx = np.argmin(vel_fit(wave[line_indx]))
ax1.plot(wave[line_indx][min_vel_indx], vel_fit(wave[line_indx][min_vel_indx]), 'kx', label='Minimum', mew=2, markersize=8)

ax1.legend(loc='lower left')
ax1.set_xlabel('Wavelength')
ax1.set_ylabel('Flux')

ax2.plot(wave[line_indx], flux[line_indx]/continuum, label='normalized flux')
pew_collection = collections.BrokenBarHCollection.span_where(wave, ymin=0, ymax=1, 
                                                        where= (wave>=wcenter-pew/2)&(wave<=wcenter+pew/2),
                                                        color='k', alpha=0.1)
ax2.add_collection(pew_collection)
ax2.set_ylim(0,1.1)
ax2.axhline(1, color='k', alpha=0.5, linestyle=':')
ax2.legend(loc='lower left')
ax2.set_xlabel('Wavelength')
ax2.set_ylabel('Normalized Flux')

Text(0,0.5,'Normalized Flux')

In [306]:
def calc_flux_variance(data, model, err_binsize):
    kernel = Box1DKernel(err_binsize)
    errors = convolve((data-model)**2, kernel, boundary=None)
    errors = errors
    errors = np.trim_zeros(errors)
    return errors

In [307]:
def test_calc_flux_variance():
    data = np.ones(10)+np.random.randn(10)
    model = np.zeros(10)
    errors = calc_flux_variance(data, model, 5)
    truth = []
    for i in range(2, 8):
        truth.append(np.sum(((data-model)**2)[i-2:i+3])/5)
    truth = np.array(truth)
    truth = np.sqrt(truth)
    assert np.isclose(truth, errors).all()
test_calc_flux_errors()

In [304]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=21
wmin = 4600
wmax = 4875

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize, visualize=False)
err_binsize = determine_error_binsize(wave, wave_bin=100)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize, absorption=True, visualize=False)
    print(good_fit_blue)
else:
    print('blue_edge_not_found')
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize, absorption=True, visualize=False)
    print(good_fit_red)
else:
    print('red_edge_not_found')
if (blue_edge_indx is not None) and (red_edge_indx is not None):
    pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=False)
    wcenter = wave[wcenter_indx]
    vel_fit = find_velocity(wave, smooth_flux, wcenter, continuum_l, continuum_r, err_binsize, visualization=True)
    line_indx = np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)]
    min_indx = int(np.floor(line_indx[0]-err_binsize/2))
    max_indx = int(np.ceil(line_indx[-1]+err_binsize/2+1))
    continuum = calc_continuum(wave[min_indx:max_indx], continuum_l, continuum_r)
    flux_var = calc_flux_variance(flux[min_indx:max_indx]-continuum,
                                    vel_fit(wave[min_indx:max_indx]), err_binsize) #These don't include the errors from the continuum subtraction yet
    plt.figure()
    continuum = calc_continuum(wave[line_indx], continuum_l, continuum_r)
    plt.errorbar(wave[line_indx], flux[line_indx]-continuum, flux_var, fmt='.-')
    plt.plot(wave[line_indx], vel_fit(wave[line_indx]))

rmse calculated over 58 points
True
rmse calculated over 58 points
True


/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:35: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:39: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:40: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# Finish Error Calculations

## Continuum error 

In [281]:
def calc_continuum_variance(wave, continuum_l, continuum_r):
    var = (1./(continuum_l.wave - continuum_r.wave))**2 * \
          ((wave - continuum_r.wave)**2 * continuum_l.error**2 + 
           (wave - continuum_l.wave)**2 * continuum_r.error**2)
    return var

In [282]:
plt.figure()
continuum = calc_continuum(wave[line_indx], continuum_l, continuum_r)
var_continuum = calc_continuum_error(wave[line_indx], continuum_l, continuum_r)
plt.plot(wave[line_indx], flux[line_indx])
plt.errorbar([continuum_l.wave, continuum_r.wave], [continuum_l.flux, continuum_r.flux], [continuum_l.error, continuum_r.error])
plt.errorbar(wave[line_indx], continuum, np.sqrt(var_continuum))

<ErrorbarContainer object of 3 artists>

## Pseudo EW error

In [338]:
def calc_pew_variance(flux, continuum, delta_wave, flux_var, continuum_var, visualize=False, wave=None):
    '''
    Calculate the variance of the equivalent width
    Inputs:
        flux: flux values over which equivalent width is calculated
        continuum: continuum values over which equivalent width is calculated
        delta_wave: the wavelength bin size (in angstroms) used in the equivalent width calculation
        flux_var: variance in the flux
        continuum_var: variance in the continuum
    Output:
        variance in the equivalent width
    '''
    pew_var_indiv = ((flux/(continuum**2)*delta_wave)**2 * continuum_var) + \
                    ((delta_wave/continuum)**2*flux_var)
    pew_var = np.sum(pew_err)
    if visualize is True:
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        if wave is not None:
            ax.errorbar(wave, (continuum-flux)/continuum, np.sqrt(pew_err))
        else:
            print('wavelength not defined')
            wave = np.arange(len(flux))
            ax.errorbar(wave,(continuum-flux)/continuum, np.sqrt(pew_err))
    return pew_var

In [339]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=21
wmin = 4600
wmax = 4875

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize, visualize=False)
err_binsize = determine_error_binsize(wave, wave_bin=100)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize, absorption=True, visualize=False)
    print(good_fit_blue)
else:
    print('blue_edge_not_found')
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize, absorption=True, visualize=False)
    print(good_fit_red)
else:
    print('red_edge_not_found')
if (blue_edge_indx is not None) and (red_edge_indx is not None):
    
    pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=False)
    wcenter = wave[wcenter_indx]
    
    vel_fit = find_velocity(wave, smooth_flux, wcenter, continuum_l, continuum_r, err_binsize, visualization=False)
    line_indx = np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)]
    min_indx = int(np.floor(line_indx[0]-err_binsize/2))
    max_indx = int(np.ceil(line_indx[-1]+err_binsize/2+1))
    continuum = calc_continuum(wave[min_indx:max_indx], continuum_l, continuum_r)
    flux_var = calc_flux_variance(flux[min_indx:max_indx]-continuum,
                                    vel_fit(wave[min_indx:max_indx]), err_binsize) #These don't include the errors from the continuum subtraction yet
    continuum = calc_continuum(wave[line_indx], continuum_l, continuum_r)
    continuum_var = calc_continuum_error(wave[line_indx], continuum_l, continuum_r)
    delta_wave = np.median(wave[1:]-wave[:-1])
    pew_var = calc_pew_variance(flux[line_indx], continuum, delta_wave, flux_var, continuum_var, visualize=True, wave=wave[line_indx])
    print(pew, np.sqrt(pew_var))
    plt.errorbar(wave[line_indx], flux[line_indx], np.sqrt(flux_var))
    plt.errorbar([continuum_l.wave, continuum_r.wave], [continuum_l.flux, continuum_r.flux], [continuum_l.error, continuum_r.error])
    plt.errorbar(wave[line_indx], continuum, np.sqrt(continuum_var))

rmse calculated over 58 points
True
rmse calculated over 58 points
True
74.0960419661 0.923418147103


/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:35: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


## velocity error

In [375]:
def calc_velocity_error(wave, flux, continuum, vel_fit, visualize=False):
    '''
    '''
    min_indx = np.argmin(vel_fit(wave))
    indx = np.argsort(wave - wave[min_indx])
    min_indx_indx = indx[indx==min_indx]  
    cum_sum_right = 0
    total = np.sum((flux-continuum))
    for i in indx[min_indx_indx:]:
        cum_sum_right += (flux-continuum)[i]
        if cum_sum_right/total > .341:
            break
    right_err = wave[i]-wave[min_indx]
    plt.axvline(wave[i])
    cum_sum_left = 0
    for j in indx[:min_indx_indx][::-1]:
        cum_sum_left += (flux-continuum)[j]
        if cum_sum_left/total > .341:
            break
    left_err = wave[min_indx]-wave[j]
    if visualize is True:
        from visualization import make_color_wheel
        colors = make_color_wheel(wave)
        plt.figure()
        for c, ind in zip(colors, indx):
            plt.plot(wave[ind], flux[ind], marker='o', ls='none', color=c)
        for k in indx[min_indx_indx:i]:
            plt.plot(wave[k], flux[k], marker='s', ls='none', color=colors[k])
        for k in indx[j:min_indx_indx]:
            plt.plot(wave[k], flux[k], marker='s', ls='none', color=colors[k])
        plt.axvline(wave[i], label='1 $\sigma$ right error')
        plt.axvline(wave[j], label='1 $\sigma$ left error')
        plt.xlabel('Wavelength')
        plt.ylabel('Flux')
        plt.legend(loc='best')
    return left_err, right_err
        

In [378]:
wave = spectrum.wave
flux = spectrum.flux
poly_deg = 2
width = 5
binsize=21
wmin = 4600
wmax = 4875

smooth_flux = smooth_signal(flux, width, poly_deg)
blue_edge_indx, wcenter_indx, red_edge_indx = find_boundary(wave, smooth_flux, wmin, wmax, binsize, visualize=True)
err_binsize = determine_error_binsize(wave, wave_bin=100)
if blue_edge_indx is not None:
    good_fit_blue, continuum_l = define_continuum(wave, smooth_flux, blue_edge_indx, binsize, err_binsize, absorption=True, visualize=True)
    print(good_fit_blue)
else:
    print('blue_edge_not_found')
if red_edge_indx is not None:
    good_fit_red, continuum_r = define_continuum(wave, smooth_flux, red_edge_indx, binsize, err_binsize, absorption=True, visualize=True)
    print(good_fit_red)
else:
    print('red_edge_not_found')
if (blue_edge_indx is not None) and (red_edge_indx is not None):
    
    pew = calc_pseudo_ew(wave, smooth_flux, continuum_l, continuum_r, visualize=True)
    wcenter = wave[wcenter_indx]
    
    vel_fit = find_velocity(wave, smooth_flux, wcenter, continuum_l, continuum_r, err_binsize, visualization=True)
    line_indx = np.arange(len(wave))[(wave>=continuum_l.wave)&(wave<=continuum_r.wave)]
    min_indx = int(np.floor(line_indx[0]-err_binsize/2))
    max_indx = int(np.ceil(line_indx[-1]+err_binsize/2+1))
    continuum = calc_continuum(wave[min_indx:max_indx], continuum_l, continuum_r)
    flux_var = calc_flux_variance(flux[min_indx:max_indx]-continuum,
                                    vel_fit(wave[min_indx:max_indx]), err_binsize) #These don't include the errors from the continuum subtraction yet
    continuum = calc_continuum(wave[line_indx], continuum_l, continuum_r)
    continuum_var = calc_continuum_error(wave[line_indx], continuum_l, continuum_r)
    delta_wave = np.median(wave[1:]-wave[:-1])
    pew_var = calc_pew_variance(flux[line_indx], continuum, delta_wave, flux_var, continuum_var, visualize=True, wave=wave[line_indx])
    print(pew, np.sqrt(pew_var))
    plt.errorbar(wave[line_indx], flux[line_indx], np.sqrt(flux_var))
    plt.errorbar([continuum_l.wave, continuum_r.wave], [continuum_l.flux, continuum_r.flux], [continuum_l.error, continuum_r.error])
    plt.errorbar(wave[line_indx], continuum, np.sqrt(continuum_var))
    vel_err = calc_velocity_error(wave[line_indx], flux[line_indx], continuum, vel_fit, visualize=True)

rmse calculated over 58 points
True
rmse calculated over 58 points
True
74.0960419661 0.923418147103


/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:35: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:39: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:40: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/bostroem/anaconda/lib/python

# Is there a misalignment in wavelengths?

In [424]:
for ifile, idir in spectra_files:
    spectrum = read_iraf_spectrum(os.path.join(idir, ifile))
    plt.plot(spectrum.wave, spectrum.flux, label=ifile)
    plt.xlim(4600, 4875)
plt.axvline(4600)
plt.axvline(4875)
plt.axvline(4900)
#plt.legend()

In [435]:
plt.figure()
for ifile, idir in spectra_files[-4:]:
    spectrum = read_iraf_spectrum(os.path.join(idir, ifile))
    plt.plot(spectrum.wave, spectrum.flux, label=ifile)
plt.axvline(5538)
plt.ylim(0, 1E-14)
plt.legend(loc='best', fontsize='x-small')

In [427]:
spectrum.flux

array([  1.09675753e-15,   1.05310452e-15,   1.16043800e-15, ...,
         2.47231818e-16,   1.98630457e-16,   2.18441512e-16], dtype=float32)